In [1]:
import re
def process_input(input):
    workflows, ratings = (p.splitlines() for p in input.split("\n\n"))
    workflows = {w[0]:[r.split(":") for r in w[1].split(",")] for w in [w[:-1].split("{") for w in workflows]}
    ratings = [{k: int(v) for k, v in dict(re.findall("([xmas])=(\d+)", r)).items()} for r in ratings]
    return workflows, ratings

In [2]:
with open("inputs/day19.txt", "r") as fn:
    input = fn.read()
    
workflows, ratings = process_input(input)

# Part 1

In [3]:
def evalulate_part(rating, workflows):
    flow_name = 'in'
    while flow_name not in ('A', 'R'):
        workflow = workflows[flow_name]
        for rule in workflow:
            if len(rule)==2:
                if eval(rule[0], rating.copy()):
                    flow_name = rule[1]
                    break
            else:
                flow_name = rule[0]
    return flow_name


In [4]:
sum_parts = 0 
for rating in ratings:
    evaluation = evalulate_part(rating, workflows)
    if evaluation == 'A':
        sum_parts += sum(rating.values())
sum_parts

492702

# Part 2

In [5]:
def split_range(range, ineq, n):
    if ineq == '<':
        tf = [range[0], min(n-1,range[1])], [max(n,range[0]), range[1]]
    else:
        tf = [max(n+1, range[0]), range[1]], [range[0], min(range[1],n)]
    
    to_ret = list()
    for e in tf:
        if e[0]>e[1]:
            to_ret.append(None)
        else:
            to_ret.append(e)
    return tuple(to_ret)

In [6]:
RANGES = [{x: [1,4000] for x in 'xmas'} | {'flow':'in'}]

while flow_names := set([r['flow'] for r in RANGES]) - {'A', 'R'}:
    for flow_name in flow_names:
        for rule in workflows[flow_name]:
            ranges_mid = [r for r in RANGES if r['flow']==flow_name]
            RANGES = [r for r in RANGES if r['flow']!=flow_name]
            if len(rule)==2:
                for ranges in ranges_mid:
                    r, ineq, n = rule[0][0], rule[0][1], int(rule[0][2:])
                    t, f = split_range(ranges[r], ineq, n)
                    
                    if t: RANGES.append(ranges | {r: t, 'flow': rule[1]})
                    if f: RANGES.append(ranges | {r: f})

            else:
                for ranges in ranges_mid:
                    RANGES.append(ranges | {'flow': rule[0]})

def flowsum(d):
    m = 1
    for x in 'xmas':
        m*=(d[x][1]-d[x][0]+1)
    return m
sum([flowsum(r) for r in RANGES if r['flow']=='A'])

138616621185978